# Vislization for Public Jobs in Brazil

In [1]:
from jinja2 import Template
from IPython.display import IFrame, HTML
import json
import pandas as pd
HTML('''
<script>
  code_show=true; 
  function code_toggle() {
   if (code_show){
     $('div.input').hide();
   } else {
     $('div.input').show();
   }
   code_show = !code_show;
  } 
  $(document).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
  <input type="submit" value="Click here to toggle on/off the raw code.">
</form>''')

In [2]:
df = pd.read_csv('D:\data\salary\monthly_salary_brazil.csv', error_bad_lines=False, warn_bad_lines=False)
def get_stats(group):
    return {'min_salary': group.min(), 'max_salary': group.max(),
            'count': group.count(), 'mean_salary': group.mean()}

job_stats = df['total_salary'].groupby(df['job']).apply(get_stats).unstack()
job_stats = job_stats.sort_values('count',ascending=False)
job_stats = job_stats[job_stats['count']>1000]
sector_stats = df['total_salary'].groupby(df['sector']).apply(get_stats).unstack()
sector_stats = sector_stats.sort_values('count',ascending=False)
sector_stats = sector_stats.iloc[:60]
def df2c3(df):
    data = {}
    categories = list(df.index.values)
    columns = []
    for col in df.columns:
        this_col = [col]
        this_col.extend(list(df[col].values))
        columns.append(this_col)
    data["columns"] = columns
    data["categories"] = categories
    return data

def df2c3_2(df):
    data = {}
    data["categories"] = list(sector_stats.index.values)
    columns = []
    for col in df.columns:
        this_col = [col]
        this_col.extend(list(sector_stats[col]))
        columns.append(this_col)
    data["columns"] = columns
    return data
sector_count=df2c3_2(sector_stats[['count', 'mean_salary']])
for i in range(1, 4):
    locals()["count_{}".format(i)] = []
    locals()["count_{}".format(i)].append("count")
    locals()["salary_{}".format(i)] = []
    locals()["salary_{}".format(i)].append("mean_salary")
    locals()["index_{}".format(i)] = []
    for j in range( (i-1)*20+1 , i*20 +1): 
        locals()["count_{}".format(i)].append(sector_count["columns"][0][j])
        locals()["salary_{}".format(i)].append(sector_count["columns"][1][j])
        locals()["index_{}".format(i)].append(sector_count["categories"][j-1])

In [3]:
for i in range(10):
    locals()["job_count_{}_{}".format(i*10, (i+1)*10)] = df2c3(job_stats[['count']].iloc[i*10:(i+1)*10].sort_values("count", ascending=False).T)
for i in range(10):
    locals()["name_list_{}_{}".format(i*10, (i+1)*10)] = [col[0] for col in locals()["job_count_{}_{}".format(i*10, (i+1)*10)]['columns']]   

In [4]:
for i in range(1,4):
    locals()["data_{}".format(i)] = {"data":{
        "columns": [locals()["count_{}".format(i)], locals()["salary_{}".format(i)]],
        "type" : "bar",
        "types" : {"mean_salary" : "spline"},
        "axes": {
            "mean_salary": 'y2'
        }    
        },
        "axis":{
                "x": {
                "type":"category",
                "categories": locals()["index_{}".format(i)]
                },
                "y2": {
                "show": "true" 
                },
                "y": {
                "label": { 
                "text": "Count",
                "position": "outer-middle"
                }
                },
                "y2": {
                "show": "true",
                "label": { 
                "text": "Mean Salary",
                "position": "outer-middle"
                }
                }        
                },
                "grid": {
                "x": {"show": "true"},
                "y": { "show": "true"}
                } 
                }

In [5]:
head = """
        <!-- Load c3.css -->
        <link href='https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.10/c3.min.css' rel='stylesheet' type='text/css'/>
        <style>.c3-line { stroke-width: 3px; }</style>
        <!-- Load d3.js and c3.js -->
        <script src='http://d3js.org/d3.v3.min.js' charset='utf-8'></script>
        <script src='http://cdnjs.cloudflare.com/ajax/libs/c3/0.4.18/c3.min.js'></script>
    """ 
template = """                                        
        <div id={{div_id}} style='width: 100%; height: 100%'></div>
        <button type='button' class='explore_distribution'>Explore Again</button>
        <script>
            var {{div_id}} = document.getElementById('{{div_id}}');
            var data_1 = {{data_1}};
            var data_2 = {{data_2}};
            var data_3 = {{data_3}};
            data_1['bindto']='#{{div_id}}'
            function draw_all(){
                var chart = c3.generate(data_1);
                console.log(data_2.data.columns)
                setTimeout(function () {
                    chart.load({
                        columns: data_2.data.columns,
                        categories: data_2.axis.x.categories             
                    });
                }, 5000);           
                setTimeout(function () {
                    chart.load({
                        columns: data_3.data.columns,
                        categories: data_3.axis.x.categories             
                    });
                }, 10000);        
                }

            draw_all()

            d3.select('.explore_distribution').on('click', function(){
                draw_all();
            });

        </script>
                                    """
iframe = '<iframe srcdoc="{source}" src="" width="{w}" height="{h}" frameborder="0" sandbox="allow-scripts"></iframe>'

In [6]:
div_id="chart"
HTML(iframe.format(
            source=Template(head + template).render(
                                                    div_id=div_id.replace(" ", "_"),
                                                    data_1=json.dumps(
                                                        data_1, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_2=json.dumps(
                                                        data_2, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_3=json.dumps(
                                                        data_3, indent=4).replace("'", "\\'").replace('"', "'")),
            w=950,
            h=400))

In [7]:
 for i in range(1, 11):
     locals()["data_{}".format(i)] =    {
                    "data": {
                        "columns": locals()["job_count_{}_{}".format((i-1)*10, i*10)]["columns"],
                        "type" : 'donut',
                    },
                    "donut": {
                        "title": "Jobs distribution"
                    }
                    }

In [8]:
head = """
        <!-- Load c3.css -->
        <link href='https://cdnjs.cloudflare.com/ajax/libs/c3/0.4.10/c3.min.css' rel='stylesheet' type='text/css'/>
        <!-- Load d3.js and c3.js -->
        <script src='http://d3js.org/d3.v3.min.js' charset='utf-8'></script>
        <script src='http://cdnjs.cloudflare.com/ajax/libs/c3/0.4.18/c3.min.js'></script>
    """ 
template = """                                        
        <div id={{div_id}} style='width: 100%; height: 100%'></div>
        <button type='button' class='explore_distribution'>Explore Again</button>
        <script>
            var {{div_id}} = document.getElementById('{{div_id}}');
            var data_1 = {{data_1}};
            var data_2 = {{data_2}};
            var data_3 = {{data_3}};
            var data_4 = {{data_4}};
            var data_5 = {{data_5}};
            var data_6 = {{data_6}};
            var data_7 = {{data_7}};
            var data_8 = {{data_8}};
            var data_9 = {{data_9}};
            var data_10 = {{data_10}};
            var list_1 = {{list_1}};
            var list_2 = {{list_2}};
            var list_3 = {{list_3}};
            var list_4 = {{list_4}};
            var list_5 = {{list_5}};
            var list_6 = {{list_6}};
            var list_7 = {{list_7}};
            var list_8 = {{list_8}};
            var list_9 = {{list_9}};
            var list_10 = {{list_10}};            
            data_1['bindto']='#{{div_id}}'
            console.log(data_1)
            function draw_all(){
                var chart = c3.generate(data_1);
                
                setTimeout(function () {
                    chart.load({
                    columns: data_2.data.columns,
                    });
                    }, 6000);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_1
                    });
                    }, 7500);
                    
                setTimeout(function () {
                    chart.load({
                    columns: data_3.data.columns,
                    });
                    }, 13500);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_2
                    });
                    }, 15000);
                    
                setTimeout(function () {
                    chart.load({
                    columns: data_4.data.columns,
                    });
                    }, 21000);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_3
                    });
                    }, 22500);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_5.data.columns,
                    });
                    }, 28500);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_4
                    });
                    }, 30000);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_6.data.columns,
                    });
                    }, 36000);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_5
                    });
                    }, 37500);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_7.data.columns,
                    });
                    }, 43500);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_6
                    });
                    }, 45000);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_8.data.columns,
                    });
                    }, 51000);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_7
                    });
                    }, 52500);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_9.data.columns,
                    });
                    }, 58500);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_8
                    });
                    }, 60000);
                                    
                setTimeout(function () {
                    chart.load({
                    columns: data_10.data.columns,
                    });
                    }, 66000);
                    
                setTimeout(function () {
                    chart.unload({
                    ids: list_9
                    });
                    }, 67500);
                                   
                }

            draw_all()

            d3.select('.explore_distribution').on('click', function(){
                draw_all();
            });

        </script>
                                    """

In [9]:
div_id="chart"
HTML(iframe.format(
            source=Template(head + template).render(
                                                    div_id=div_id.replace(" ", "_"),
                                                    data_1=json.dumps(
                                                        data_1, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_2=json.dumps(
                                                        data_2, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_3=json.dumps(
                                                        data_3, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_4=json.dumps(
                                                        data_4, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_5=json.dumps(
                                                        data_5, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_6=json.dumps(
                                                        data_6, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_7=json.dumps(
                                                        data_7, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_8=json.dumps(
                                                        data_8, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_9=json.dumps(
                                                        data_9, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    data_10=json.dumps(
                                                        data_10, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_1=json.dumps(
                                                        name_list_0_10, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_2=json.dumps(
                                                        name_list_10_20, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_3=json.dumps(
                                                        name_list_20_30, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_4=json.dumps(
                                                        name_list_30_40, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_5=json.dumps(
                                                        name_list_40_50, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_6=json.dumps(
                                                        name_list_50_60, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_7=json.dumps(
                                                        name_list_60_70, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_8=json.dumps(
                                                        name_list_70_80, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_9=json.dumps(
                                                        name_list_80_90, indent=4).replace("'", "\\'").replace('"', "'"),
                                                    list_10=json.dumps(
                                                        name_list_90_100, indent=4).replace("'", "\\'").replace('"', "'")),
                                                    w=950,
                                                    h=400))